<a href="https://colab.research.google.com/github/Oec8cek/Ai-Understanding/blob/main/0914_%ED%8C%90%EA%B2%B0%EB%AC%B8_pdf_%EB%8B%A4%EC%9A%B4%EB%A1%9C%EB%93%9C_%26_xlsx_%EB%B3%80%ED%99%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap
# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500
Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300
Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.KodjwKO3bQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.EZtYnmOkOh/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.nKVJhwZnwt/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from time import sleep
from bs4 import BeautifulSoup
import requests
import re
import os
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm

# Dataframe
import pandas as pd

# 2023.08.02 셀레니움 라이브러리 수정사항 반영
# https://stackoverflow.com/questions/76428561/typeerror-webdriver-init-got-multiple-values-for-argument-options
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=service, options=options)

# 판결문 크롤링

In [ ]:
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
prefs = {"download.default_directory" : "/content/drive/MyDrive/Colab Notebooks/EWHA/[2023-2] 통학론/통학론 팀플/dataset/판결문pdf",
           "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True}
options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=service, options=options)

url = "https://www.scourt.go.kr/search/total/TotalSearch.work"
driver.get(url)

In [ ]:
# 지정한 CSS 선택자를 사용하여 요소 찾기
btn_menu_all = driver.find_element(By.CSS_SELECTOR, "#category > option:nth-child(7)")
#category
#category > option:nth-child(7)
btn_menu_all.click()

#un = driver.find_element(By.CSS_SELECTOR, "body > div:nth-child(2) > div.header > div.gnbOuter > div > div.allMenu > ul > li:nth-child(2) > ul > li:nth-child(1) > a")
#un.click()
#살인과 관련된 pdf
element=driver.find_element(By.ID, 'dpWord')
element.send_keys("살인")
element.send_keys("\n")
time.sleep(2)

In [ ]:
import shutil

In [ ]:
file_number = 0
for page in range(1, 210): # 1page~209page
  if page ==1: pass
  elif page % 5 == 1: num = driver.find_element(By.CLASS_NAME, "prev2").click()
  else:
    if page % 5 == 0: n = 7
    else: n = page % 5 + 2
    num = driver.find_element(By.CSS_SELECTOR, "#tab7 > div.pagelist > a:nth-child({})".format(n)).click()
  for m in range(3,22,2):
    dun = driver.find_element(By.CSS_SELECTOR, "#tab7 > div:nth-child({}) > a:nth-child(3)".format(m))
    dun.click()
    time.sleep(1)
    # 가장 최근에 변경된 파일의 파일명 변경
    new_file_name = max([download_path + "\\" + f for f in os.listdir(download_path)],key=os.path.getctime)
    shutil.move(new_file_name,os.path.join(download_path, str(file_number)))


filepath = 'c:\\downloads'
filename = max([filepath + '\\' + f for f in os.listdir(filepath)], key=os.path.getctime)
shutil.move(os.path.join(filepath, filename), new_filename)


#1: #tab7 > div.pagelist > a.selected
#4: #tab7 > div.pagelist > a:nth-child(6)
#5: tab7 > div.pagelist > a:nth-child(7)
#6: #tab7 > div.pagelist > a.prev2

#7: #tab7 > div.pagelist > a:nth-child(4)
# 9: #tab7 > div.pagelist > a:nth-child(6)


KeyboardInterrupt: ignored

In [ ]:
pdf_folder_path = "/content/drive/MyDrive/Colab Notebooks/EWHA/[2023-2] 통학론/통학론 팀플/dataset/판결문pdf"
pdf_filename_list = os.listdir(pdf_folder_path)

# xlsx 변환

In [ ]:
import os
from PyPDF2 import PdfReader
import pandas as pd

# PDF 파일이 있는 디렉토리 경로를 지정합니다.
pdf_directory = r'C:\Users\yeo05\Desktop\pdfpan'

# Excel 파일로 저장할 경로를 지정합니다.
excel_file_path = '판결내용_1.xlsx'

# PDF 디렉토리의 모든 파일을 확인합니다.
all_data = []  # 모든 PDF 파일의 데이터를 저장할 리스트

for filename in os.listdir(pdf_directory):
    if filename.endswith('.pdf'):
        pdf_file_path = os.path.join(pdf_directory, filename)

        # PDF 파일 열기
        pdf_reader = PdfReader(pdf_file_path)

        # 각 페이지의 텍스트를 추출하여 리스트에 추가합니다.
        pdf_data = []
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            pdf_data.append(page_text)

        # 파일 이름과 데이터를 튜플로 묶어 리스트에 추가합니다.
        all_data.append((filename, "\n\n".join(pdf_data)))

# 데이터를 DataFrame으로 변환합니다.
df = pd.DataFrame(all_data, columns=["판결문", "내용"])

# DataFrame을 Excel 파일로 저장합니다.
df.to_excel(excel_file_path, index=False, engine='openpyxl')

print(f"모든 PDF 파일의 텍스트를 '{excel_file_path}' 파일에 Excel 형식으로 저장했습니다.")

